# H5T0_MLP: implement an MLP using PyTorch for image classification

### create your "signature":
#### Run the cell but do NOT modify the cell
#### The signature needs to be displayed right below the cell in this file

In [1]:
import platform, socket, getpass
from time import time
from datetime import datetime
import random
signature_for_mlp=[socket.gethostbyname(socket.gethostname()),
                   getpass.getuser(),
                   datetime.now(),
                   random.random()]
print(signature_for_mlp) 

['192.168.50.217', 'sloanatkins', datetime.datetime(2025, 12, 9, 14, 46, 32, 245606), 0.1616925553497467]


In [ ]:
#sample code to save model and signature
#torch.save({"model":model.state_dict(),
#            "signature":signature_for_mlp},
#            "MLP_best.pt") 

### This is a complete application of image classification using an MLP
steps: \
define an MLP \
define a function for training the MLP \
define a function for testing the MLP \
define a function for saving the model and your `signature_for_mlp` \
load data \
train the MLP in a for loop using the training set and the validation set\
evalaute the best model on the test set\
display accuracy and confusion matrix for each of the three sets: train, validation, and test sets
### you may use the code in lecture nodes
### write your code below this line

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import torch.serialization

torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------------------
# LOAD DATA (correct keys)
# ---------------------------
data = torch.load("H5T0_train_val_test_data.pt", weights_only=False)

X_train = data["X_train"]
y_train = data["Y_train"]
X_val   = data["X_val"]
y_val   = data["Y_val"]
X_test  = data["X_test"]
y_test  = data["Y_test"]

# ---------------------------
# RESHAPE + CONVERT TO TENSORS
# ---------------------------
X_train = torch.tensor(X_train.reshape(len(X_train), -1), dtype=torch.float32)
X_val   = torch.tensor(X_val.reshape(len(X_val), -1), dtype=torch.float32)
X_test  = torch.tensor(X_test.reshape(len(X_test), -1), dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.long)
y_val   = torch.tensor(y_val, dtype=torch.long)
y_test  = torch.tensor(y_test, dtype=torch.long)

# ---------------------------
# DATA LOADERS
# ---------------------------
train_ds = TensorDataset(X_train, y_train)
val_ds   = TensorDataset(X_val, y_val)
test_ds  = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=256, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=256, shuffle=False)

# ---------------------------
# DEFINE MLP MODEL
# ---------------------------
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(256, 10)
        )

    def forward(self, x):
        return self.model(x)

model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ---------------------------
# TRAINING ONE EPOCH
# ---------------------------
def train_one_epoch(model, loader):
    model.train()
    total_loss = 0
    for X, y in loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        preds = model(X)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# ---------------------------
# EVALUATION FUNCTION
# ---------------------------
def evaluate(model, loader):
    model.eval()
    preds_list = []
    labels_list = []

    with torch.no_grad():
        for X, y in loader:
            X = X.to(device)
            out = model(X)
            preds = torch.argmax(out, dim=1).cpu()
            preds_list.append(preds)
            labels_list.append(y)

    preds = torch.cat(preds_list)
    labels = torch.cat(labels_list)
    acc = accuracy_score(labels, preds)
    cm = confusion_matrix(labels, preds)
    return acc, cm

# ---------------------------
# TRAIN LOOP — TRACK BEST MODEL
# ---------------------------
best_val_acc = -1
best_state = None

num_epochs = 20

for epoch in range(num_epochs):
    loss = train_one_epoch(model, train_loader)
    train_acc, _ = evaluate(model, train_loader)
    val_acc, _ = evaluate(model, val_loader)

    print(f"Epoch {epoch+1}/{num_epochs} | Loss={loss:.4f} | Train Acc={train_acc:.4f} | Val Acc={val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_state = model.state_dict().copy()

print("\nBest validation accuracy =", best_val_acc)

# ---------------------------
# SAVE BEST MODEL WITH SIGNATURE
# ---------------------------
torch.save({
    "model": best_state,
    "signature": signature_for_mlp
}, "MLP_best.pt")

print("Saved best model as MLP_best.pt")

# ---------------------------
# LOAD BEST MODEL
# ---------------------------
best_model = MLP().to(device)
best_model.load_state_dict(best_state)

# ---------------------------
# FINAL EVALUATION
# ---------------------------
train_acc, train_cm = evaluate(best_model, train_loader)
val_acc, val_cm = evaluate(best_model, val_loader)
test_acc, test_cm = evaluate(best_model, test_loader)

print("\n=== FINAL ACCURACIES ===")
print("Train:", train_acc)
print("Val:", val_acc)
print("Test:", test_acc)

print("\n=== CONFUSION MATRICES ===")
print("Train:\n", train_cm)
print("\nVal:\n", val_cm)
print("\nTest:\n", test_cm)


/var/folders/fb/bmnkg3m57kb_ck5_xxc75zfh0000gn/T/ipykernel_47564/3007412319.py:9: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])


Using device: cpu
Epoch 1/20 | Loss=1.9347 | Train Acc=0.5790 | Val Acc=0.5580
Epoch 2/20 | Loss=1.1602 | Train Acc=0.6310 | Val Acc=0.6200
Epoch 3/20 | Loss=0.9201 | Train Acc=0.6650 | Val Acc=0.6570
Epoch 4/20 | Loss=0.8258 | Train Acc=0.7410 | Val Acc=0.6990
Epoch 5/20 | Loss=0.7161 | Train Acc=0.7560 | Val Acc=0.7220
Epoch 6/20 | Loss=0.6730 | Train Acc=0.7830 | Val Acc=0.7390
Epoch 7/20 | Loss=0.5981 | Train Acc=0.8180 | Val Acc=0.7700
Epoch 8/20 | Loss=0.5467 | Train Acc=0.8360 | Val Acc=0.7860
Epoch 9/20 | Loss=0.5051 | Train Acc=0.8520 | Val Acc=0.7890
Epoch 10/20 | Loss=0.4855 | Train Acc=0.8390 | Val Acc=0.7650
Epoch 11/20 | Loss=0.4512 | Train Acc=0.8770 | Val Acc=0.8050
Epoch 12/20 | Loss=0.4186 | Train Acc=0.8750 | Val Acc=0.8050
Epoch 13/20 | Loss=0.3986 | Train Acc=0.8850 | Val Acc=0.7940
Epoch 14/20 | Loss=0.3813 | Train Acc=0.8670 | Val Acc=0.7880
Epoch 15/20 | Loss=0.3894 | Train Acc=0.8770 | Val Acc=0.8060
Epoch 16/20 | Loss=0.3538 | Train Acc=0.8940 | Val Acc=0.8240